# 时间与孕周数据处理

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [102]:
男胎检测数据=pd.read_csv('男胎检测数据.csv')
男胎检测数据.head()

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,Y染色体浓度,X染色体浓度,13号染色体的GC含量,18号染色体的GC含量,21号染色体的GC含量,被过滤掉读段数的比例,染色体的非整倍体,怀孕次数,生产次数,胎儿是否健康
0,1,A001,31,160.0,72.0,2023/2/1,自然受孕,20230429,1,11w+6,...,0.025936,0.038061,0.377069,0.389803,0.399399,0.027484,NaN,1,0,是
1,2,A001,31,160.0,73.0,2023/2/1,自然受孕,20230531,2,15w+6,...,0.034887,0.059572,0.371542,0.384771,0.391706,0.019617,NaN,1,0,是
2,3,A001,31,160.0,73.0,2023/2/1,自然受孕,20230625,3,20w+1,...,0.066171,0.075995,0.377449,0.390582,0.399480,0.022312,NaN,1,0,是
3,4,A001,31,160.0,74.0,2023/2/1,自然受孕,20230716,4,22w+6,...,0.061192,0.052305,0.375613,0.389251,0.397212,0.023280,NaN,1,0,是
4,5,A002,32,149.0,74.0,2023/11/9,自然受孕,20240219,1,13w+6,...,0.059230,0.059708,0.380260,0.393618,0.404868,0.024212,NaN,2,1,否


In [103]:
print(男胎检测数据.末次月经.describe())

count           1070
unique           190
top       2022-12-28
freq              32
Name: 末次月经, dtype: object


In [104]:
# 将['末次月经', '检测日期']转为datetime类型
男胎检测数据['末次月经'] = pd.to_datetime(男胎检测数据['末次月经'], errors='coerce')
男胎检测数据['检测日期'] = pd.to_datetime(男胎检测数据['检测日期'].astype(str), format='%Y%m%d', errors='coerce')
男胎检测数据[['末次月经', '检测日期']].apply(lambda col: col.dt.date)
男胎检测数据[['末次月经', '检测日期']].head()


,末次月经,检测日期
0,2023-02-01,2023-04-29
1,2023-02-01,2023-05-31
2,2023-02-01,2023-06-25
3,2023-02-01,2023-07-16
4,2023-11-09,2024-02-19


In [105]:
孕周数据=男胎检测数据.检测孕周
孕周数据

0       11w+6
1       15w+6
2       20w+1
3       22w+6
4       13w+6
        ...  
1077    17w+5
1078    11w+4
1079    12w+4
1080    13w+4
1081    14w+4
Name: 检测孕周, Length: 1082, dtype: object

In [106]:
import re

def convert_gestational_age(age_str):
    if pd.isna(age_str):
        return np.nan
    match = re.match(r'(\d+)w\+?(\d+)?', age_str)
    if match:
        weeks, days = match.groups()
        return int(weeks) + (int(days) / 7 if days else 0)
    else:
        return np.nan  # 或处理无效格式

孕周数据_数值 = 孕周数据.apply(convert_gestational_age)
孕周数据_数值 = 孕周数据_数值.round(2)
孕周数据_数值
男胎检测数据['检测孕周']=孕周数据_数值

In [107]:
男胎检测数据.检测孕周

0       11.86
1       15.86
2       20.14
3       22.86
4       13.86
        ...  
1077    17.71
1078    11.57
1079    12.57
1080    13.57
1081    14.57
Name: 检测孕周, Length: 1082, dtype: float64

# 身体指标计算

In [108]:
print(男胎检测数据.孕妇BMI.isna().sum())

0


# 缺失值与异常值

In [109]:
mask = 男胎检测数据[['Y染色体浓度','检测孕周','孕妇BMI']].isna().any(axis=1)
男胎检测数据.loc[mask]

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,Y染色体浓度,X染色体浓度,13号染色体的GC含量,18号染色体的GC含量,21号染色体的GC含量,被过滤掉读段数的比例,染色体的非整倍体,怀孕次数,生产次数,胎儿是否健康


In [111]:
男胎检测数据.to_csv('男胎检测数据_预处理后.csv',index=False)